# Stock Predict Model

## Import Modules

In [34]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# For Graph
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Yahoo API
import yfinance as yf

## Utility Functions

In [35]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()
    return preds

## Dataset

In [36]:
class Stock_Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        self.y = y if y is None else torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Neural Network Model

In [37]:
class Stock_Model(nn.Module):
    def __init__(self, input_dim):
        super(Stock_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.Sigmoid(),
            nn.Linear(16, 8),
            nn.Sigmoid(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

## Feature Selection

In [38]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

## Configurations

In [39]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
config = {
    'seed': 666999,
    'select_all': True,
    'valid_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 64,
    'learning_rate': 1e-5,
    'early_stop': 400,
    'save_path': './models/stock.ckpt'
}

## Training Loop

In [40]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)
        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Dataloader

In [41]:
same_seed(config['seed'])
# re-index
index = ["Open", "High", "Low", "Close", "Volume", "Adj Close"]
# train_data size: 5255 x 7 (id + 6 features From 2000-01-01 to 2020-12-31)
# test_data size: 577 x 7 (id + 6 features From 2021-01-01 to 2023-12-31)
train_data, test_data = pd.DataFrame(yf.download('2330.TW', start='2000-01-01', end='2020-12-31'))[index].values, \
                        pd.DataFrame(yf.download('2330.TW', start='2021-01-01', end='2023-12-31'))[index[:-1]].values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = Stock_Dataset(x_train, y_train), \
                                             Stock_Dataset(x_valid, y_valid), \
                                             Stock_Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
train_data size: (4204, 6)
valid_data size: (1051, 6)
test_data size: (578, 5)
number of features: 5


## Training

In [42]:
model = Stock_Model(input_dim=x_train.shape[1]).to(device)

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU],
    schedule = torch.profiler.schedule(
        wait=0,
        warmup=0,
        active=1
    ),
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
) as prof:
    trainer(train_loader, valid_loader, model, config, device)
    prof.step()

/opt/homebrew/lib/python3.11/site-packages/torch/profiler/profiler.py:268: UserWarning: Profiler won't be using warmup, this can skew profiler results
  warn("Profiler won't be using warmup, this can skew profiler results")
STAGE:2023-05-25 21:15:08 81634:13942186 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
Epoch [1/3000]: 100%|██████████| 66/66 [00:00<00:00, 155.76it/s, loss=7.29e+3]


Epoch [1/3000]: Train loss: 15108.6053, Valid loss: 14529.1760
Saving model with loss 14529.176...


Epoch [2/3000]: 100%|██████████| 66/66 [00:00<00:00, 151.16it/s, loss=7.82e+3]


Epoch [2/3000]: Train loss: 14447.3489, Valid loss: 13178.6697
Saving model with loss 13178.670...


Epoch [3/3000]: 100%|██████████| 66/66 [00:00<00:00, 166.53it/s, loss=1.58e+4]


...

Epoch [591/3000]: 100%|██████████| 66/66 [00:00<00:00, 205.46it/s, loss=1.15e+4]


Epoch [591/3000]: Train loss: 7850.0411, Valid loss: 7637.2798


Epoch [592/3000]: 100%|██████████| 66/66 [00:00<00:00, 197.55it/s, loss=1.04e+4]
STAGE:2023-05-25 21:19:23 81634:13942186 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-05-25 21:19:26 81634:13942186 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


Epoch [592/3000]: Train loss: 7844.9147, Valid loss: 7676.5408

Model is not improving, so we halt the training session.


STAGE:2023-05-25 21:36:00 81634:13942186 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-05-25 21:36:00 81634:13942186 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-05-25 21:36:00 81634:13942186 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [46]:
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))


-----------------  ------------  ------------  ------------  ------------  ------------  ------------  
             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------  ------------  ------------  ------------  ------------  ------------  ------------  
    ProfilerStep*       100.00%       1.629ms       100.00%       1.629ms       1.629ms             1  
-----------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 1.629ms



In [49]:
%tensorboard --logdir=./log/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

## Plot learning curves with tensorboard

In [51]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

## Testing

In [45]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = Stock_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

100%|██████████| 10/10 [00:00<00:00, 45.09it/s]
